In [1]:
import sys
sys.path.append('..')
from src.utilities import *

## 2. Master creation

### 2.1 Define cross join of month - region - brand

In [21]:
sales_train = pd.read_csv(os.path.join(raw_path, 'sales_train.csv'))

months = pd.DataFrame({'month': sales_train.month.unique()})
regions = pd.DataFrame({'region': sales_train.region.unique()})
brands = pd.DataFrame({'brand': ['brand_1', 'brand_2']})

months['dummy_col'] = 0
regions['dummy_col'] = 0
brands['dummy_col'] = 0

master = months.merge(regions, how = 'outer', on = 'dummy_col')
master = master.merge(brands, how = 'outer', on = 'dummy_col')
master.drop(columns = 'dummy_col', inplace = True)
master = master.merge(sales_train, how = 'left', on = ['month', 'region', 'brand'])
master

,month,region,brand,sales
0,2020-01,region_0,brand_1,0.0
1,2020-01,region_0,brand_2,0.0
2,2020-01,region_1,brand_1,0.0
3,2020-01,region_1,brand_2,0.0
4,2020-01,region_2,brand_1,0.0
...,...,...,...,...
8035,2021-08,region_198,brand_2,NaN
8036,2021-08,region_199,brand_1,NaN
8037,2021-08,region_199,brand_2,NaN
8038,2021-08,region_200,brand_1,NaN


### 2.2 Append features

In [ ]:


region_features = extract_region_features()
brand_features = ..





master = master.join(region_features, how = 'left', on = ['month', 'region', 'brand'])

### 2.3 Save master table

In [ ]:
master.to_csv(os.path.join(processed_path, 'master.csv'))